In [1]:
# Import modules
import pandas as pd
import numpy as np
import bert
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import  Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tqdm import tqdm
import matplotlib.pyplot as plt

# string processing
import re
from keras.utils.np_utils import to_categorical
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing, feature_selection, metrics

print("TensorFlow Version:",tf.__version__)
print("Hub version: ",hub.__version__)
pd.set_option('display.max_colwidth',1000)

TensorFlow Version: 2.2.0
Hub version:  0.8.0


### Start here:

In [2]:
# to navigate to the data location
import os

# get current directory 
path = os.getcwd() 

# parent directory
parent = os.path.dirname(path)

df_merge_quality = pd.read_csv(parent + '/data/US_patent_abstract_5000_2015_with_title_1_5y.csv')
df_merge_quality.shape

(5000, 30)

In [3]:
df = df_merge_quality[['claims_text', 'quality_rank']]
df

,claims_text,quality_rank
0,"What is claimed is: \n \n 1. An invitation information push method comprising:\n after receiving an invitation request sent by a microblog user, a server sending invitation information to clients corresponding to a number of invited users carried in the invitation request, wherein the invited users are users who have not registered microblog, and the number of the invited users N is greater than or equal to 1; and \n upon receiving the invitation information, each client creating an invitation information guide to guide the user who has not registered microblog to register microblog; \n wherein the server sending the invitation information to the clients corresponding to the invited users comprises: \n determining, by the server, whether one or more of the invited users carried in the invitation request are in a restricted list; and \n if none of the invited users carried in the invitation request is in the restricted list, sending the invitation information to the client...",0
1,"What is claimed is: \n \n 1. An implant connector comprising:\n a first member including a first cavity that defines a longitudinal axis along which a first implant is configured for disposal in a fixed orientation; and \n a second member connected to the first member along a first axis and defining a second cavity, the second cavity comprising a U-shaped configuration with an open top portion that faces away from the first member and defines a longitudinal axis along which a second implant is configured for disposal, the second cavity being rotatable about a second axis that is disposed in a transverse orientation and intersecting the longitudinal axis of the second cavity such that the second implant is rotatable in a first plane relative to the first implant. \n \n \n \n 2. An implant connector according to claim 1 , wherein the first member further includes a lateral opening in communication with the first cavity and disposed for receiving the first imp...",0
2,"What is claimed is: \n \n 1. A spearfishing device, the device comprising:\n an elongate barrel including a first planar end, a second planar end, a first surface, a second surface, a third surface, and a fourth surface, the barrel including a length along a shooting axis and a width across the shooting axis with the length being greater than the width, the barrel defining a spear conduit along the shooting axis and two band conduits that are parallel to the spear conduit, each of the spear conduit and the two band conduits extending entirely within the barrel from the first planar end to the second planar end, and each of the first, second, third, and fourth surfaces extending from the first planar end to the second planar end. \n \n \n \n 2. The device of claim 1 , wherein the spear conduit is coaxial with the shooting axis. \n \n \n 3. The device of claim 2 , wherein the spear conduit is located between the two band conduits. \n \n ...",1
3,"1. A computer-implemented method for the automated selection and loading of media items from a computer into a media presentation device, the method comprising:\n at the time of initiating the removal and loading process,\n receiving a media item current playlist from the media presentation device; \n receiving current user taste data associated with a user of the media presentation device, the taste data reflecting recent use of the presentation device by the said user; \n generating a recommended playlist responsive to the user taste data; \n ranking the media items in the recommended playlist responsive to the user taste data; \n determining an amount of memory storage space available on the media presentation device for storing new items; and \n automatically removing as many media items from the media presentation device as necessary to free memory space sufficient to accommodate at least the highest ranked one of the ranked media items, wherein the media items are removed fro...",1
4,What is claimed is: \n \n 1. A

In [4]:
df['claims_text'] = df['claims_text'].apply(str)

/home/lin_menghsien/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [5]:
df.apply(lambda x: sum(x.isnull()), axis=0)

claims_text     0
quality_rank    0
dtype: int64

In [6]:
# Functions for constructing BERT Embeddings: input_ids, input_masks, input_segments and Inputs
MAX_SEQ_LEN=512 # max sequence length

def get_masks(tokens):
    """Masks: 1 for real tokens and 0 for paddings"""
    return [1]*len(tokens) + [0] * (MAX_SEQ_LEN - len(tokens))
 
def get_segments(tokens):
    """Segments: 0 for the first sequence, 1 for the second"""  
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (MAX_SEQ_LEN - len(tokens))

def get_ids(tokens, tokenizer):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens,)
    input_ids = token_ids + [0] * (MAX_SEQ_LEN - len(token_ids))
    return input_ids

def create_single_input(sentence, tokenizer, max_len):
    """Create an input from a sentence"""
    stokens = tokenizer.tokenize(sentence)
    stokens = stokens[-max_len:] # last n words
    stokens = ["[CLS]"] + stokens + ["[SEP]"]
 
    ids = get_ids(stokens, tokenizer)
    masks = get_masks(stokens)
    segments = get_segments(stokens)

    return ids, masks, segments
 
def convert_sentences_to_features(sentences, tokenizer):
    """Convert sentences to features: input_ids, input_masks and input_segments"""
    input_ids, input_masks, input_segments = [], [], []
 
    for sentence in tqdm(sentences,position=0, leave=True):
      ids,masks,segments=create_single_input(sentence,tokenizer,MAX_SEQ_LEN-2)
      assert len(ids) == MAX_SEQ_LEN
      assert len(masks) == MAX_SEQ_LEN
      assert len(segments) == MAX_SEQ_LEN
      input_ids.append(ids)
      input_masks.append(masks)
      input_segments.append(segments)

    return [np.asarray(input_ids, dtype=np.int32), 
          np.asarray(input_masks, dtype=np.int32), 
          np.asarray(input_segments, dtype=np.int32)]

def create_tonkenizer(bert_layer):
    """Instantiate Tokenizer with vocab"""
    vocab_file=bert_layer.resolved_object.vocab_file.asset_path.numpy()
    do_lower_case=bert_layer.resolved_object.do_lower_case.numpy() 
    tokenizer=bert.bert_tokenization.FullTokenizer(vocab_file,do_lower_case)
    return tokenizer

In [7]:
def bert_model(callable_object):
    # Load the pre-trained BERT base model
    bert_layer = hub.KerasLayer(handle=callable_object, trainable=True)  
   
    # BERT layer three inputs: ids, masks and segments
    input_ids = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="input_ids")           
    input_masks = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="input_masks")       
    input_segments = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="segment_ids")
    
    inputs = [input_ids, input_masks, input_segments] # BERT inputs
    pooled_output, sequence_output = bert_layer(inputs) # BERT outputs
    
    # Add a hidden layer
    x = Dense(units=768, activation='relu')(pooled_output)
    x = Dropout(0.1)(x)
 
    # Add output layer
    outputs = Dense(2, activation="sigmoid")(x)

    # Construct a new model
    model = Model(inputs=inputs, outputs=outputs)
    return model

model = bert_model("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1")
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_ids[0][0]                  
                                                                 input_masks[0][0]            

In [8]:
df['quality_rank'] = df['quality_rank'].astype(int)

/home/lin_menghsien/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
# Create examples for training and testing
# df = df.sample(frac=1) # Shuffle the dataset  # already shuffle in the csv file
tokenizer = create_tonkenizer(model.layers[3])
X_train = convert_sentences_to_features(df['claims_text'][:4000], tokenizer)
X_test = convert_sentences_to_features(df['claims_text'][4000:], tokenizer)


# for binary classification, since in the loss function I use binary_crossentropy, looks like prefer this single array
y_train = to_categorical(df['quality_rank'][:4000].values)
y_test =  to_categorical(df['quality_rank'][4000:].values)




100%|██████████| 1000/1000 [00:17<00:00, 58.16it/s]


In [10]:
# distribution of label value in train
np.unique([np.argmax(i) for i in y_train], return_counts=True)

(array([0, 1]), array([2397, 1603]))

In [11]:
# distribution of label value in test
np.unique([np.argmax(i) for i in y_test], return_counts=True)

(array([0, 1]), array([595, 405]))

In [12]:
# setup checkpoint

checkpoint_path = "ckpt_bert_fine_tune_last_n_words/"

ckpt = tf.train.Checkpoint(model = model) # https://www.tensorflow.org/api_docs/python/tf/train/Checkpoint

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=2)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

Latest checkpoint restored!!


In [13]:
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        ckpt_manager.save()
        print("Checkpoint saved at {}.".format(checkpoint_path))

In [ ]:
# Train the model
BATCH_SIZE = 10
EPOCHS = 4

# Use Adam optimizer to minimize the categorical_crossentropy loss
opt = Adam(learning_rate=2e-5)
model.compile(optimizer=opt, 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Fit the data to the model
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    verbose = 1,
                    callbacks=[MyCustomCallback()])


Epoch 1/4
400/400 [==============================] - 7852s 20s/step - loss: 0.6370 - accuracy: 0.6467 - val_loss: 0.5847 - val_accuracy: 0.6890
Epoch 2/4
400/400 [==============================] - 9351s 23s/step - loss: 0.5908 - accuracy: 0.6892 - val_loss: 0.6109 - val_accuracy: 0.6890
Epoch 3/4
180/400 [============>.................] - ETA: 1:18:14 - loss: 0.5104 - accuracy: 0.7589

In [14]:
# disconnected, re-start training from epoch = 3

BATCH_SIZE = 10
EPOCHS = 2


# Use Adam optimizer to minimize the categorical_crossentropy loss
opt = Adam(learning_rate=2e-5)
model.compile(optimizer=opt, 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Fit the data to the model
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    verbose = 1,
                    callbacks=[MyCustomCallback()])


Epoch 1/2
400/400 [==============================] - 5427s 14s/step - loss: 0.5214 - accuracy: 0.7477 - val_loss: 0.6674 - val_accuracy: 0.6540
Epoch 2/2
400/400 [==============================] - 5329s 13s/step - loss: 0.3776 - accuracy: 0.8395 - val_loss: 0.7390 - val_accuracy: 0.6510


In [15]:
pred_test = model.predict(X_test)
pred_test

array([[0.7840179 , 0.21532494],
       [0.43648085, 0.57667387],
       [0.29131627, 0.72529626],
       ...,
       [0.6094679 , 0.4348665 ],
       [0.03140494, 0.968728  ],
       [0.2674314 , 0.7325458 ]], dtype=float32)

In [16]:
np.savetxt('Predict_Output/BERT_fine_tune_5yr_claims_dev_prob_last_n_words.csv', pred_test)

In [17]:
predicted = [np.argmax(pred) for pred in 
             pred_test]

In [18]:
y_test_binary = df['quality_rank'][4000:].values
y_test_binary

array([0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,

In [19]:
## Accuracy, Precision, Recall
accuracy = metrics.accuracy_score(y_test_binary, predicted)
auc = metrics.roc_auc_score(y_test_binary, predicted)  # predicted_prob), check doc, seems the second argument required to be shape (n_samples,) for binary case 
                            #multi_class="ovr") # check documentation and seems "ovr" not good for only binary target class
print("Accuracy:",  round(accuracy,3))
print("Auc:", round(auc,3))
print("Detail:")
print(metrics.classification_report(y_test_binary, predicted))

# Accuracy output 0.627 => align with model performance result!

Accuracy: 0.651
Auc: 0.621
Detail:
              precision    recall  f1-score   support

           0       0.68      0.78      0.73       595
           1       0.59      0.46      0.52       405

    accuracy                           0.65      1000
   macro avg       0.63      0.62      0.62      1000
weighted avg       0.64      0.65      0.64      1000



In [12]:
# let's restore the 2nd epoch

checkpoint_path = "ckpt_bert_fine_tune_last_n_words/"

ckpt = tf.train.Checkpoint(model = model) # https://www.tensorflow.org/api_docs/python/tf/train/Checkpoint

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=2)

if ckpt_manager.latest_checkpoint:
    ckpt.restore('ckpt_bert_fine_tune_last_n_words/ckpt-2')
    print("Latest checkpoint restored!!")

Latest checkpoint restored!!


In [13]:
pred_test = model.predict(X_test)
pred_test

array([[0.6688505 , 0.34002274],
       [0.49800253, 0.48948818],
       [0.48060858, 0.51535887],
       ...,
       [0.38333338, 0.60166514],
       [0.27293935, 0.6813065 ],
       [0.57289016, 0.43524444]], dtype=float32)

In [14]:
np.savetxt('Predict_Output/BERT_fine_tune_5yr_claims_dev_prob_last_n_words.csv', pred_test)

In [15]:
predicted = [np.argmax(pred) for pred in 
             pred_test]

In [16]:
y_test_binary = df['quality_rank'][4000:].values
#y_test_binary

In [17]:
## Accuracy, Precision, Recall
accuracy = metrics.accuracy_score(y_test_binary, predicted)
auc = metrics.roc_auc_score(y_test_binary, predicted)  # predicted_prob), check doc, seems the second argument required to be shape (n_samples,) for binary case 
                            #multi_class="ovr") # check documentation and seems "ovr" not good for only binary target class
print("Accuracy:",  round(accuracy,3))
print("Auc:", round(auc,3))
print("Detail:")
print(metrics.classification_report(y_test_binary, predicted))

# Accuracy output 0.627 => align with model performance result!

Accuracy: 0.689
Auc: 0.666
Detail:
              precision    recall  f1-score   support

           0       0.72      0.79      0.75       595
           1       0.64      0.54      0.59       405

    accuracy                           0.69      1000
   macro avg       0.68      0.67      0.67      1000
weighted avg       0.68      0.69      0.68      1000

